<a href="https://colab.research.google.com/github/hepuliu/deep_learning/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN for Stock Prediction

# Data Processing

In [13]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
# import training set
from google.colab import drive
drive.mount
dataset_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/Course_DL/Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

In [15]:
# feature scaling - normalization between 0 to 1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [16]:
# create a data structure with 60 past timesteps and 1 output
x_train = []
y_train = []
for i in range(60, 1258):
  x_train.append(training_set_scaled[i-60:i, 0])
  y_train.append(training_set_scaled[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [17]:
# reshaping - adding indicator
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


# Building the RNN

In [18]:
# importing keras libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# initializing the RNN
regressor = Sequential()

# adding first LSTM layer and some dropout regularization
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# adding the second LSTM layer and some dropout regularization
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# adding the third LSTM layer and some dropout regularization
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# adding the fourth LSTM layer and some dropout regularization
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# adding the output layer of the LSTM
regressor.add(Dense(units = 1))

# compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# fitting the RNN to the training set
regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)



Epoch 1/100
38/38 [==============================] - 13s 127ms/step - loss: 0.0407
Epoch 2/100
38/38 [==============================] - 5s 125ms/step - loss: 0.0065
Epoch 3/100
38/38 [==============================] - 5s 130ms/step - loss: 0.0059
Epoch 4/100
38/38 [==============================] - 5s 131ms/step - loss: 0.0048
Epoch 5/100
38/38 [==============================] - 5s 130ms/step - loss: 0.0048
Epoch 6/100
38/38 [==============================] - 6s 154ms/step - loss: 0.0049
Epoch 7/100
38/38 [==============================] - 5s 133ms/step - loss: 0.0044
Epoch 8/100
38/38 [==============================] - 5s 127ms/step - loss: 0.0043
Epoch 9/100
38/38 [==============================] - 5s 126ms/step - loss: 0.0044
Epoch 10/100
38/38 [==============================] - 5s 125ms/step - loss: 0.0044
Epoch 11/100
38/38 [==============================] - 5s 125ms/step - loss: 0.0044
Epoch 12/100
38/38 [==============================] - 5s 121ms/step - loss: 0.0045
Epoch 13/100

# Making Prediction

# Result Visualization